In [1]:
import tensorflow as tf
#import tensorflow.nn.rnn_cell as rnn
import numpy as np
#import tensorflow.keras as keras
#from tensorflow.keras.initializers import Constant
import tensorflow.keras.backend as K
from tensorflow.python.platform import gfile

from tqdm import tqdm
import os


In [5]:
model_dir = './models/100hz/data_shuffle_7_30_19/realLSTM_data_shuffle_U256_i199_mse1.52_524.69.pb'

In [11]:
sess = tf.Session()
print("load graph")
with gfile.FastGFile(model_dir,'rb') as f:
    graph_def = tf.GraphDef()
graph_def.ParseFromString(f.read())
sess.graph.as_default()
tf.import_graph_def(graph_def, name='')
graph_nodes=[n for n in graph_def.node]
names = []
for t in graph_nodes:
    names.append(t.name)
print(names)
print('graph loaded.')

load graph
['input_tensor', 'input_state', 'wf', 'wf/read', 'wc', 'wc/read', 'wo', 'wo/read', 'uf', 'uf/read', 'uc', 'uc/read', 'uo', 'uo/read', 'bf', 'bf/read', 'bc', 'bc/read', 'bo', 'bo/read', 'weights1', 'weights1/read', 'weights2', 'weights2/read', 'bias1', 'bias1/read', 'bias2', 'bias2/read', 'split/split_dim', 'split', 'MatMul_1', 'MatMul_2', 'MatMul_3', 'Add_3', 'MatMul_5', 'Add_4', 'Const_3', 'Const_4', 'Mul_1', 'Add_5', 'clip_by_value_1/Minimum/y', 'clip_by_value_1/Minimum', 'clip_by_value_1/y', 'clip_by_value_1', 'Mul_2', 'Add_6', 'MatMul_6', 'Add_7', 'Tanh', 'output_c', 'Add_8', 'MatMul_7', 'Add_9', 'Const_5', 'Const_6', 'Mul_3', 'Add_10', 'clip_by_value_2/Minimum/y', 'clip_by_value_2/Minimum', 'clip_by_value_2/y', 'clip_by_value_2', 'Tanh_1', 'output_h', 'MatMul_8', 'Add_11', 'Relu', 'MatMul_9', 'output_forRelu', 'output', 'output_state/axis', 'output_state']
graph loaded.


In [22]:
data_x=np.loadtxt('./TapData/data_raw_100Hz.csv',delimiter=",")
data_x=data_x[:,(6-3):6].reshape((len(data_x),3))

In [27]:
data_x=np.loadtxt('./TapData/data_100Hz_shuffle.csv',delimiter=",")
data_x=data_x[:,(6-3):6].reshape((len(data_x),3))

In [28]:
num_units=256
data_state = np.zeros([1,num_units*2],dtype='float')
predictY=np.zeros([len(data_x)])

for i in tqdm(range(len(data_x))):
    feed={'input_tensor:0':data_x[i:i+1],
          'input_state:0':data_state,
            #y:data_y[i:i+1],
          #loss_weight_in:loss_weight[i:i+1]
           }

    data_state,_predictY = sess.run(['output_state:0','output:0'],feed_dict=feed)
    #print(data_h.shape)
    #data_c = sess.run(c,feed_dict=feed)
    predictY[i]=_predictY[0]

In [30]:
np.savetxt('TF_predictY_100hz_shuffle.csv',predictY)

In [26]:
np.savetxt('TF_predictY_100hz_None_shuffle.csv',predictY)